In [1]:
import json
import re
from zs4procext.parser import ParametersParser, ActionsParser, SchemaParser, KeywordSearching, DimensionlessParser
from zs4procext.actions import Chemical, Actions, CollectLayer, Triturate, Sonicate, Recrystallize, Partition,Degas, Purify, Wait, Wash, Quench, Reflux, Extract, Filter, MakeSolution, Add, Stir, SetTemperature, Concentrate, PhaseSeparation, DrySolution, DrySolid
from zs4procext.prompt import PromptFormatter
from zs4procext.evaluator import Evaluator

In [5]:
with open("resources/Mistral-7B-Instruct_inference_example.json", "r") as f:
    examples = json.load(f)
txt = examples["examples"][1]["prompt"]
print(txt)
ph_reg = r"(\b[pP][hH]\b)[\D\s=]*(\+?-?\d+\b(\.?,?\d+)?\b-*\b(\d*\.?,?\d+)?\b)"
parser = ParametersParser()
parser.model_post_init(None)
matches = parser.get_parameters(txt)
for match in matches:
    print(match)

"""parser = ActionsParser(text=txt)
parser.initialize()
matches = parser.parse(txt)
i = 0
for action in matches["actions"]:
    print(f"Action: {action}")
    print(f"Content: {repr(matches['content'][i])}")
    i += 1"""


You are an expert on organic synthesis.
Take your time and extract information as truthfully as possible using the provided context. If any information is not provided or you are unsure, use 'N/A'.
Create an action graph from the following paragraph:
'To a solution of 20 (390.9 g, 1.11 mol) and SbCl3 (7.62 g, 0.0334 mol) in anhydrous dichloromethane (2.27 L) at −15° C., diethylamino sulfurtrifluoride (DAST) (258 ml, 2.0 mol) was added drop-wise. The reaction mixture was stirred for 3 hours at −20° C. and then allowed to warm up to room temperature overnight. The reaction was quenched with 1N NaOH solution until pH=3-4 at −20° C., and then solid NaHCO3 was added to pH=8. The organic layer was separated and the aqueous solution was extracted with dichloromethane. The combined organic extracts were washed with water, brine, dried over Na2SO4 and concentrated to provide 330 g of the desired product as a brown oil.'
To create the action graph use only some the following actions, adding all 

'parser = ActionsParser(text=txt)\nparser.initialize()\nmatches = parser.parse(txt)\ni = 0\nfor action in matches["actions"]:\n    print(f"Action: {action}")\n    print(f"Content: {repr(matches[\'content\'][i])}")\n    i += 1'

In [2]:
with open("resources/Llama-70B_response_chemicals.json", "r") as f:
    examples = json.load(f)

string = '{"chemical" : "N/A", "amount": 7 mL, "dropwise": True}'
atributes = ["chemical", "dropwise"]
schema_parser = SchemaParser(atributes_list=atributes)
print(string)
schema_parser.model_post_init(None)
quantity_parser = ParametersParser(time=False, temperature=False, pressure=False)
quantity_parser.model_post_init(None)
results = schema_parser.parse_schema(string)
print(results)

{"chemical" : "N/A", "amount": 7 mL, "dropwise": True}
['{"chemical" : "N/A", "amount": 7 mL, "dropwise": True}']


In [3]:
microwave_keywords = ["microwave", "microwaves"]
organic_keywords = ["organic", "org", "hydrophobic"]
aqueous_keywords = ["aqueous", "aq", "hydrophilic", "water", "aquatic"]
filtrate_keywords = ["filtrate", "lixiviate", "percolate", "permeate", "liquid"]
precipitate_keywords = ["precipitate", "residue", "filter cake", "sludge", "solid"]
ph_parser = KeywordSearching(keywords_list=["ph"])
ph_parser.model_post_init(None)
microwave_parser = KeywordSearching(keywords_list=microwave_keywords)
microwave_parser.model_post_init(None)
organic_parser = KeywordSearching(keywords_list=organic_keywords)
aqueous_parser = KeywordSearching(keywords_list=aqueous_keywords)
organic_parser.model_post_init(None)
aqueous_parser.model_post_init(None)
filtrate_parser = KeywordSearching(keywords_list=filtrate_keywords)
precipitate_parser = KeywordSearching(keywords_list=precipitate_keywords)
filtrate_parser.model_post_init(None)
precipitate_parser.model_post_init(None)
context = "#"
schemas = ['{"chemical" : "water", "amount": 7 mL, "dropwise": True}', '{"chemical" : "N/A", "amount": 7 mL, "dropwise": True}']
atributes = ["chemical", "dropwise"]
schema_parser = SchemaParser(atributes_list=atributes)
schema_parser.model_post_init(None)
quantity_parser = ParametersParser(time=False, temperature=False, pressure=False)
quantity_parser.model_post_init(None)
conditions_parser = ParametersParser()
conditions_parser.model_post_init(None)
actions = Add.generate_action(context, schemas, schema_parser, quantity_parser, conditions_parser, ph_parser)
#actions = MakeSolution.generate_action(context, schemas, schema_parser, quantity_parser, conditions_parser)
#actions = Stir.generate_action(context, conditions_parser)
#actions = SetTemperature.generate_action(context, conditions_parser, microwave_parser)
#actions = Concentrate.generate_action(context)
#actions = Extract.generate_action(context, schemas, schema_parser, quantity_parser)
#actions = DrySolution.generate_action(context, schemas, schema_parser, quantity_parser)
#actions = Quench.generate_action(context, schemas, schema_parser, quantity_parser, conditions_parser)
#actions = Wait.generate_action(context, conditions_parser)
#actions = PhaseSeparation.generate_action(context)
#actions = CollectLayer.generate_action(context, aqueous_parser, organic_parser)
#actions = Reflux.generate_action(context, conditions_parser)
#actions = Wash.generate_action(context, schemas, schema_parser, quantity_parser)
#actions = DrySolid.generate_action(context, conditions_parser)
#actions = Filter.generate_action(context, filtrate_parser, precipitate_parser)
#actions = Purify.generate_action(context)
#actions = Recrystallize.generate_action(context, schemas, schema_parser, quantity_parser, conditions_parser)
#actions = Degas.generate_action(context, conditions_parser)
#actions = Partition.generate_action(context, schemas, schema_parser, quantity_parser)
#actions = Sonicate.generate_action(context, conditions_parser)
#actions = Triturate.generate_action(context, schemas, schema_parser, quantity_parser)
for action in actions:
    print(action)


water
N/A
{'action': 'Add', 'content': {'material': {'name': 'water', 'quantity': ['7.0 milliliter']}, 'dropwise': True, 'temperature': None, 'atmosphere': None, 'duration': None}}


In [3]:
dataset_path = "resources/evaluation_test.txt"
evaluator: Evaluator = Evaluator(reference_dataset_path=dataset_path)
actions_eval = evaluator.evaluate_actions(dataset_path)
sequence_eval = evaluator.evaluate_actions_order(dataset_path)

1
1
0
0
0
1


In [6]:
print(Stir.type)

onlyconditions


In [5]:
list1 = [1,2,3,4,5,6,7]
threshold = 3
i = 5
print(list1[:i - threshold + 1] + list1[i + 1:])

[1, 2, 3, 7]
